# 👟 Behaviors

In [ ]:
if __name__ == "__main__":
    %pip install -q -r requirements.txt

In [ ]:
import random

import ipywidgets as W
import networkx as nx
import numpy as np

from ipyforcegraph.behaviors import NodeSelection
from ipyforcegraph.forcegraph import ForceGraph

In [ ]:
fg = ForceGraph(behaviors=[NodeSelection()])
fg

In [ ]:
atlas = nx.graph_atlas_g()

In [ ]:
def update_graph(i: int = None) -> None:
    G = atlas[i if i is not None else random.randint(0, len(atlas))]
    nodes = list(G.nodes)
    fg.source.nodes.array = np.array(nodes, dtype=np.int32)
    fg.source.links.array = np.array(
        [
            [nodes.index(source), nodes.index(target)]
            for source, target in G.edges
        ],
        dtype=np.int32
    )

In [ ]:
@W.interact
def g(i=(0, len(atlas) - 1)):
    update_graph(i)

The data can be set from the kernel.

In [ ]:
fg.behaviors[0].value.array = np.array([1, 2, 3], dtype=np.int32)